# Verden er mer enn SAS og CSV

Til daglig leser vi mest SAS-datasett, litt CSV og litt relasjonelle databaser. Det finnes dog så veldig mye mer, og Pandas har mulighet til å lese veldig mye mer enn hva SAS klarer.

Pandas kommer ikke med noe eget filformat, men det finnes et mylder av ulike filer og databaser som er åpent tilgjengelig, og som kan brukes både av Pandas, R, og flere andre språk.

Her går vi gjennom en del av de vanligste opsjonene til å lese CSV, og en del av de ulike andre datakildene vi kan lese med Pandas.


## CSV-opsjoner

Typiske opsjoner vi trenger når vi leser CSV er:
- spesifisere skilletegn: `sep` argumentet
- spesifisere header: `header=False` (default True, men False når names er angitt)
- hoppe over X antall rader: `skiprows` (default 0)
- angi variabelnavn: `names` (leser default fra første rad når `header=True`)
- angi kolonnetyper: `dtype` (default `None`, forsøker å gjette, men kan angi i form av en `dict`)

Den fulle listen av opsjoner finnes på https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

In [1]:
import pandas as pd
import numpy as np

In [2]:
varnames = ["NAME", "DEPARTMENT_NAME", "TITLE", "REGULAR", "RETRO", "OTHER", "OVERTIME", "INJURED", "DETAIL", "QUINN", "TOTAL", "POSTAL"]
dtypes = ['str', 'str', 'str', 'float', 'float', 'float', 'float', 'float', 'float', 'float', 'float', 'str']

Vi kunne i prinsippet laget en dictionary med variabelnavn og typer som vi kunne brukt som argument til `dtype`, men med trøblete filer kan det være mer praktisk å parse tall etterpå.

In [3]:
dtype_obj = {vname: dt for vname, dt in zip(varnames, dtypes)}

In [4]:
dtype_obj

{'NAME': 'str',
 'DEPARTMENT_NAME': 'str',
 'TITLE': 'str',
 'REGULAR': 'float',
 'RETRO': 'float',
 'OTHER': 'float',
 'OVERTIME': 'float',
 'INJURED': 'float',
 'DETAIL': 'float',
 'QUINN': 'float',
 'TOTAL': 'float',
 'POSTAL': 'str'}

In [5]:
floats = [v for v, t in zip(varnames, dtypes) if t=='float']

In [6]:
pay = pd.read_csv('/data/kurs/python/payroll/employeeearningscy18full.csv', 
                  encoding='latin1', dtype='str', names=varnames, skiprows=1)

In [7]:
pay.head()

,NAME,DEPARTMENT_NAME,TITLE,REGULAR,RETRO,OTHER,OVERTIME,INJURED,DETAIL,QUINN,TOTAL,POSTAL
0,"Williams,David C.",Boston Police Department,Police Officer,"95,184.68",NaN,"503,025.78","45,176.88","5,307.62","24,610.00","4,375.69","677,680.65",02356
1,"Earley,John W",Boston Police Department,Police Lieutenant,"26,130.54","13,001.47","204,120.61",668.36,NaN,440.00,"6,532.63","250,893.61",02131
2,"Bucelewicz,Helen",Boston Police Department,Police Officer,"8,676.89",NaN,"200,000.00",NaN,NaN,NaN,NaN,"208,676.89",02135
3,"Evans,William B",Boston Police Department,Commissioner (Bpd),"149,999.93","3,461.56","165,858.44",NaN,NaN,NaN,NaN,"319,319.93",02127
4,"Jones,Ronnie C",Boston Police Department,Police Officer,"462,149.98",NaN,"114,973.46",NaN,NaN,NaN,NaN,"577,123.44",02126


In [8]:
for val in floats:
    pay[val] = pd.to_numeric(pay[val].apply(lambda x: str(x).replace(',', '')), errors='coerce')

In [9]:
pay.dtypes

NAME                object
DEPARTMENT_NAME     object
TITLE               object
REGULAR            float64
RETRO              float64
OTHER              float64
OVERTIME           float64
INJURED            float64
DETAIL             float64
QUINN              float64
TOTAL              float64
POSTAL              object
dtype: object

In [10]:
pay.head()

,NAME,DEPARTMENT_NAME,TITLE,REGULAR,RETRO,OTHER,OVERTIME,INJURED,DETAIL,QUINN,TOTAL,POSTAL
0,"Williams,David C.",Boston Police Department,Police Officer,95184.68,NaN,503025.78,45176.88,5307.62,24610.0,4375.69,677680.65,02356
1,"Earley,John W",Boston Police Department,Police Lieutenant,26130.54,13001.47,204120.61,668.36,NaN,440.0,6532.63,250893.61,02131
2,"Bucelewicz,Helen",Boston Police Department,Police Officer,8676.89,NaN,200000.00,NaN,NaN,NaN,NaN,208676.89,02135
3,"Evans,William B",Boston Police Department,Commissioner (Bpd),149999.93,3461.56,165858.44,NaN,NaN,NaN,NaN,319319.93,02127
4,"Jones,Ronnie C",Boston Police Department,Police Officer,462149.98,NaN,114973.46,NaN,NaN,NaN,NaN,577123.44,02126


## Posisjonsbestemte filer

Alle arkivfilene våre er såkalt posisjonsbestemte, hvor hver variabel begynner på en angitt posisjon på linjen. Disse formatene er sjeldne å komme over i den virkelige verden, men Pandas har en metode for å lese disse: `read_fwf`.

Vi har en liten demo-fil, som ser ut som følger:

In [11]:
!cat data/fixed_width_demo.fwf

123abcd 456.34
423FGLFF  0.01

In [12]:
fwf_dtypes = {'A': 'float', 'B': 'str', 'C': 'float'}

In [13]:
import pandas as pd
df = pd.read_fwf('../test.fwf', widths=[3, 5, 6], skiprows=0, names=['A', 'B', 'C'], dtype=fwf_dtypes)

In [14]:
df

,A,B,C
0,123.0,abcd,456.34
1,423.0,FGLFF,0.01


In [15]:
df.dtypes

A    float64
B     object
C    float64
dtype: object

## SQL

Foreløpig har vi ikke Oracle SQL-drivere på maskinen så vi klarer ikke å lese oracle-databaser. Men vi kan lese sqlite med `sqlite3` biblioteket: https://docs.python.org/3.5/library/sqlite3.html

In [16]:
import sqlite3
import pandas as pd

In [17]:
cnx = sqlite3.connect('./data/main.db')

In [18]:
df1 = pd.read_sql_query("SELECT * FROM trainingdata LIMIT 100", con=cnx)

### Finne innholdet i en database

Om du snubler over en sqlite-database og vil finne ut hva den inneholder, er det mulig å lese `sqlite_master` tabellen. Om databasen er stor er det kanskje ikke så hensiktsmessig å printe hele sånn vi gjør her, men vi kan likevel spørre f.eks. hvilke tabeller som finnes. Legg også merke til at indekser listes her.

In [19]:
pd.read_sql_query("SELECT * FROM sqlite_master", con=cnx)

,type,name,tbl_name,rootpage,sql
0,table,trainingdata,trainingdata,2,"CREATE TABLE ""trainingdata"" (\n""index"" INTEGER..."
1,index,ix_trainingdata_index,trainingdata,3,"CREATE INDEX ""ix_trainingdata_index""ON ""traini..."


## HDF5

HDF5 er et mindre kjent, men åpent og ofte nyttig binært filformat for tabulære datasett. En fil kan inneholde mer enn bare ett datasett, noe som gir store muligheter til å lagre f.eks. metadata sammen med datasettet.

For å finne hvilke datasett en fil inneholder, kan vi lage et `HDFStore` objekt som vi kan undersøke.

In [20]:
store = pd.HDFStore("data/trainingdata.hd5")

In [21]:
store.keys()

['/data']

In [22]:
df = store.get("data")

In [23]:
store.close()

Eller:

In [24]:
df = pd.read_hdf("data/trainingdata.hd5", key="data")

## Prøv selv

Om du ikke kommer på ting å prøve selv, kan dette være noen ideer:

- Lag en ny sqlite database, og lag en ny tabell hvor du setter inn noen verdier fra et selvvalgt datasett.
- Filen `fagkoder.txt` er en posisjonsbestemt arkivfil. Fildefinisjonen/metadata slik den ligger i datadok er lagret i `fagkoder.ddok` Lag et program som først leser metadataene, og så leser arkivfilen inn til en dataframe.